In [1]:
import csv
import os
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from PIL import Image

%matplotlib inline


def load_itk_image(filename):
    with open(filename) as f:
        contents = f.readlines()
        line = [k for k in contents if k.startswith("TransformMatrix")][0]
        transform = np.array(line.split(" = ")[1].split(" ")).astype("float")
        transform = np.round(transform)
        if np.any(transform != np.array([1, 0, 0, 0, 1, 0, 0, 0, 1])):
            isflip = True
        else:
            isflip = False
    itkimage = sitk.ReadImage(filename)
    numpyimage = sitk.GetArrayFromImage(itkimage)
    if isflip == True:
        numpyimage = numpyimage[:, ::-1, ::-1]
    numpyOrigin = np.array(list(itkimage.GetOrigin()))  # CT原点坐标
    numpySpacing = np.array(list(itkimage.GetSpacing()))  # CT像素间隔
    return numpyimage, numpyOrigin, numpySpacing


def worldToVoxelCoord(worldCoord, origin, spacing):
    stretchedVoxelCoord = np.absolute(worldCoord - origin)
    voxelCoord = stretchedVoxelCoord / spacing
    return voxelCoord


def normalizePlanes(npzarray):
    maxHU = 400.0
    minHU = -1000.0
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray[npzarray > 1] = 1.0
    npzarray[npzarray < 0] = 0.0
    return npzarray


#########################################################
def truncate_hu(image_array):
    image_array[image_array > 400] = 0
    image_array[image_array < -1000] = 0


def normalization(image_array):
    ImgArrmax = image_array.max()
    ImgArrmin = image_array.min()
    image_array = (image_array - ImgArrmin) / (ImgArrmax - ImgArrmin)
    avg = image_array.mean()
    image_array = image_array - avg
    return image_array

In [82]:
img_path = "data/subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896761494371822656720.mhd"
npimg1, nporg1, npspc1 = load_itk_image(img_path)
truncate_hu(npimg1)
# image_array = normalization(npimg1)
npimg1

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [30]:
def readCSV(filename):
    lines = []
    with open(filename, "r") as f:
        csvreader = csv.reader(f)
        for line in csvreader:
            lines.append(line)
    return lines


anno_path = "data/CSVFILES/annotations.csv"
annos = readCSV(anno_path)
cand = annos[24]

worldCoord = np.asarray([float(cand[1]), float(cand[2]), float(cand[3])])
voxelCoord = worldToVoxelCoord(worldCoord, nporg1, npspc1)

print(worldCoord)
print(voxelCoord)

[-100.5679445    67.26051683 -231.816619  ]
[128.04205092 344.30087843  41.3573488 ]
